In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [2]:
from ktrain import text

In [3]:
aspirational_df = pd.read_excel('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/aspirational_plus_batch_1_batch_2_merged.xlsx')
aspirational_df

,sentence,label,phrase,batch,original_label,changed?
0,why am i here?,0,"[""ever since i was little i wanted to be a doc...",batch_1,0,False
1,well why does anyone pursue a higher education?,0,"[""ever since i was little i wanted to be a doc...",batch_1,0,False
2,to better one self and be able to succeed late...,1,"[""ever since i was little i wanted to be a doc...",batch_1,1,False
3,ever since i was little i wanted to be a docto...,1,"[""ever since i was little i wanted to be a doc...",batch_1,1,False
4,i always wanted to be able to help people and ...,1,"[""ever since i was little i wanted to be a doc...",batch_1,1,False
...,...,...,...,...,...,...
8715,"anyways, the path im referring to is basically...",0,['one of these goals is being in a career i en...,batch_2,0,False
8716,i know that i want a career in which ill help ...,0,['one of these goals is being in a career i en...,batch_2,0,False
8717,"in any case, i intend to keep moving forward i...",0,['one of these goals is being in a career i en...,batch_2,0,False
8718,i am here to fulfill a covenant with myself an...,1,['i am here to fulfill a covenant with myself ...,batch_2,1,False


In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
file_path = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/aspirational_plus_batch_1_batch_2_merged.xlsx"
df = pd.read_excel(file_path)

# Ensure 'batch' column is a string
df["batch"] = df["batch"].astype(str)

# Split Batch 1 only
batch1_df = df[df["batch"] == "batch_1"]

# Perform train-test split for Batch 1 (keeping stratification)
train_batch1, test_batch1 = train_test_split(batch1_df, test_size=0.22, random_state=42, stratify=batch1_df["label"])

# Verify counts
print(f"Batch 1 Train: {len(train_batch1)}, Test: {len(test_batch1)}, Total: {len(batch1_df)}")

# Ensure all sentences in Batch 1 train/test are also in Batch 1 + Batch 2
batch1_2_df = df  # Full dataset (Batch 1 + Batch 2)

# --- 🛠️ **Fix for incorrect filtering** ---
# Step 1: Retrieve all Batch 1 sentences from Batch 1+2
train_batch1_2 = batch1_2_df[batch1_2_df["sentence"].isin(train_batch1["sentence"])]
test_batch1_2 = batch1_2_df[batch1_2_df["sentence"].isin(test_batch1["sentence"])]

# Step 2: Add the remaining Batch 2 sentences (not already in train/test)
batch2_df = df[df["batch"] == "batch_2"]
train_batch1_2 = pd.concat([train_batch1_2, batch2_df], ignore_index=True)

# Verify counts match
print(f"Batch 1 + Batch 2 Train: {len(train_batch1_2)}, Test: {len(test_batch1_2)}, Total: {len(batch1_2_df)}")

# # Save final DataFrames
train_batch1.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/train_batch1.xlsx", index=False)
test_batch1.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/test_batch1.xlsx", index=False)
train_batch1_2.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/train_batch1_2.xlsx", index=False)
test_batch1_2.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/test_batch1_2.xlsx", index=False)

print("✅ Train and test sets successfully corrected and saved!")

Batch 1 Train: 2727, Test: 770, Total: 3497
Batch 1 + Batch 2 Train: 8116, Test: 909, Total: 8720
✅ Train and test sets successfully corrected and saved!


In [9]:
MAXLEN = 150

In [41]:
X = train_batch1['sentence']
y = train_batch1['original_label']

In [42]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [43]:
model_name = 'distilbert-base-uncased'

In [44]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

preprocessing train...
language: en
train sequence lengths:
	mean : 20
	95percentile : 39
	99percentile : 54


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 41
	99percentile : 55


In [45]:
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)

In [46]:
distillbert_learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
364/364 [==============================] - 91s 222ms/step - loss: 0.4547 - accuracy: 0.7955 - val_loss: 0.4150 - val_accuracy: 0.8059
Epoch 2/5
364/364 [==============================] - 87s 235ms/step - loss: 0.3022 - accuracy: 0.8840 - val_loss: 0.4319 - val_accuracy: 0.8077
Epoch 3/5
364/364 [==============================] - 94s 257ms/step - loss: 0.2147 - accuracy: 0.9271 - val_loss: 0.4699 - val_accuracy: 0.7949
Epoch 4/5
364/364 [==============================] - 96s 263ms/step - loss: 0.1322 - accuracy: 0.9569 - val_loss: 0.5380 - val_accuracy: 0.7967
Epoch 5/5
364/364 [==============================] - 93s 253ms/step - loss: 0.0782 - accuracy: 0.9762 - val_loss: 0.6728 - val_accuracy: 0.8022


In [55]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

18/18 [==============================] - 7s 234ms/step
              precision    recall  f1-score   support

           0       0.88      0.85      0.87       409
           1       0.59      0.67      0.63       137

    accuracy                           0.80       546
   macro avg       0.74      0.76      0.75       546
weighted avg       0.81      0.80      0.81       546



array([[346,  63],
       [ 45,  92]])

In [56]:
distillbert_learner.model.summary()

Model: "tf_distil_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_79 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [58]:
distillbert_test_data = test_batch1['sentence'].tolist()
distillbert_test_label = test_batch1['original_label'].tolist()

In [59]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [60]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [61]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 503, False Positive: 67, False Negative: 70, True Positive: 130


In [62]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.88      0.88       570
           1       0.66      0.65      0.65       200

    accuracy                           0.82       770
   macro avg       0.77      0.77      0.77       770
weighted avg       0.82      0.82      0.82       770
 



In [31]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.91      0.90       582
           1       0.70      0.66      0.68       188

    accuracy                           0.85       770
   macro avg       0.80      0.79      0.79       770
weighted avg       0.85      0.85      0.85       770
 



In [24]:
#distillbert_predictor.save('./model/distilbert_base_uncased_model') # 256 MB

In [38]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.8009703180009733
